<a href="https://colab.research.google.com/github/dalismo/Finance_News_Sentiment/blob/ai%2Fworking_on_count_vector/AI_v2_count_vector_ml_sentiment_analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [717 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.lau

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("StopWords").getOrCreate()
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/sentiment-train.csv'
# df = pd.read_csv(url,encoding='latin-1', header=None)
# url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/yelp_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sentiment-train.csv"), sep=",", header=None)

# Show DataFrame
df.show(5, truncate=False)

+--------+-----------------------------------------------------------------------------------------------------+
|_c0     |_c1                                                                                                  |
+--------+-----------------------------------------------------------------------------------------------------+
|Positive|Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                              |
|Positive|Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                     |
|Positive|RSX: Russian Equities Are Still Undervalued                                                          |
|Positive|XLB: Hold Your Position                                                                              |
|Positive|Musgrave Minerals broad, near-surface high-grade results open up further Cue gold discovery potential|
+--------+--------------------------------------------------------------------------------------

In [4]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['_c1']))
data_df.show()

+--------+--------------------+------+
|     _c0|                 _c1|length|
+--------+--------------------+------+
|Positive|Autodesk Is Down ...|    71|
|Positive|Blue Shield of Ca...|    80|
|Positive|RSX: Russian Equi...|    43|
|Positive|XLB: Hold Your Po...|    23|
|Positive|Musgrave Minerals...|   101|
|Positive|ViacomCBS to rest...|    49|
|Positive|Oil climbs to one...|    51|
|Negative|Australian regula...|    67|
|Positive|The Wall Street J...|    90|
|Positive|Ora Banda Mining ...|    50|
|Positive|Google Pay Plans ...|    43|
|Negative|Toyota, Honda Ang...|    69|
| Neutral|ZY Investor Alert...|    44|
| Neutral|Calima Energy spu...|    58|
|Positive|These Marijuana S...|    56|
|Positive|Moderna ETFs to R...|    61|
| Neutral|MacroGenics Annou...|    72|
| Neutral|Zymeworks Announc...|   173|
| Neutral|BioXcel Therapeut...|   130|
| Neutral|Merck to Present ...|    80|
+--------+--------------------+------+
only showing top 20 rows



In [5]:
df2 = data_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
df2.printSchema()
df2.show(20, truncate = False)


root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|class   |text                                                                                                                                                                         |length|
+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Positive|Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                                                                                                      |71    |
|Positive|Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                        

In [6]:
df2.createOrReplaceTempView('temp')
df3 = spark.sql('select case class when "Positive" then 1.0  when "Negative" then 2.0 else 0 end as label, text from temp')
df3.show(20, truncate = False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|text                                                                                                                                                                         |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1.0  |Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                                                                                                      |
|1.0  |Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                                                                                             |
|1.0  |RSX: Russian Equities Are Still Undervalued                                        

In [7]:
from pyspark.ml.feature import  Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(df3)
wordsData.show()

+-----+--------------------+--------------------+
|label|                text|               words|
+-----+--------------------+--------------------+
|  1.0|Autodesk Is Down ...|[autodesk, is, do...|
|  1.0|Blue Shield of Ca...|[blue, shield, of...|
|  1.0|RSX: Russian Equi...|[rsx:, russian, e...|
|  1.0|XLB: Hold Your Po...|[xlb:, hold, your...|
|  1.0|Musgrave Minerals...|[musgrave, minera...|
|  1.0|ViacomCBS to rest...|[viacomcbs, to, r...|
|  1.0|Oil climbs to one...|[oil, climbs, to,...|
|  2.0|Australian regula...|[australian, regu...|
|  1.0|The Wall Street J...|[the, wall, stree...|
|  1.0|Ora Banda Mining ...|[ora, banda, mini...|
|  1.0|Google Pay Plans ...|[google, pay, pla...|
|  2.0|Toyota, Honda Ang...|[toyota,, honda, ...|
|  0.0|ZY Investor Alert...|[zy, investor, al...|
|  0.0|Calima Energy spu...|[calima, energy, ...|
|  1.0|These Marijuana S...|[these, marijuana...|
|  1.0|Moderna ETFs to R...|[moderna, etfs, t...|
|  0.0|MacroGenics Annou...|[macrogenics, ann...|


In [8]:
from pyspark.ml.feature import CountVectorizer
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
model = count.fit(wordsData)
featurizedData = model.transform(wordsData)
featurizedData.show()

+-----+--------------------+--------------------+--------------------+
|label|                text|               words|         rawFeatures|
+-----+--------------------+--------------------+--------------------+
|  1.0|Autodesk Is Down ...|[autodesk, is, do...|(11788,[6,9,19,37...|
|  1.0|Blue Shield of Ca...|[blue, shield, of...|(11788,[2,7,177,5...|
|  1.0|RSX: Russian Equi...|[rsx:, russian, e...|(11788,[48,262,63...|
|  1.0|XLB: Hold Your Po...|[xlb:, hold, your...|(11788,[158,335,1...|
|  1.0|Musgrave Minerals...|[musgrave, minera...|(11788,[26,49,89,...|
|  1.0|ViacomCBS to rest...|[viacomcbs, to, r...|(11788,[0,34,1891...|
|  1.0|Oil climbs to one...|[oil, climbs, to,...|(11788,[0,7,100,1...|
|  2.0|Australian regula...|[australian, regu...|(11788,[5,175,179...|
|  1.0|The Wall Street J...|[the, wall, stree...|(11788,[1,7,292,4...|
|  1.0|Ora Banda Mining ...|[ora, banda, mini...|(11788,[0,5,72,32...|
|  1.0|Google Pay Plans ...|[google, pay, pla...|(11788,[75,217,26...|
|  2.0

In [9]:
from pyspark.ml.feature import  IDF
#IDF down-weighs features which appear frequently in a corpus. This generally improves performance when using text as features since most frequent, and hence less important words, get down-weighed.
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "features").show()  
# We want only the label and features columns for our machine learning models

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(11788,[6,9,19,37...|
|  1.0|(11788,[2,7,177,5...|
|  1.0|(11788,[48,262,63...|
|  1.0|(11788,[158,335,1...|
|  1.0|(11788,[26,49,89,...|
|  1.0|(11788,[0,34,1891...|
|  1.0|(11788,[0,7,100,1...|
|  2.0|(11788,[5,175,179...|
|  1.0|(11788,[1,7,292,4...|
|  1.0|(11788,[0,5,72,32...|
|  1.0|(11788,[75,217,26...|
|  2.0|(11788,[2,65,230,...|
|  0.0|(11788,[43,44,63,...|
|  0.0|(11788,[10,83,338...|
|  1.0|(11788,[0,3,13,18...|
|  1.0|(11788,[0,7,172,2...|
|  0.0|(11788,[11,12,13,...|
|  0.0|(11788,[1,3,5,11,...|
|  0.0|(11788,[0,2,3,12,...|
|  0.0|(11788,[0,4,12,13...|
+-----+--------------------+
only showing top 20 rows



In [10]:
seed = 0  # set seed for reproducibility
trainDF, testDF = rescaledData.randomSplit([0.8,0.2],seed)

In [11]:
trainDF.count()

3965

In [12]:
testDF.count()

1035

In [13]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np
lr = LogisticRegression(maxIter = 10)

paramGrid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, np.linspace(0.3, 0.01, 10)) \
    .addGrid(lr.elasticNetParam, np.linspace(0.3, 0.8, 6)) \
    .build()
crossval_lr = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid_lr,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds= 5)  
cvModel_lr = crossval_lr.fit(trainDF)
best_model_lr = cvModel_lr.bestModel.summary
best_model_lr.predictions.columns

KeyboardInterrupt: ignored

In [14]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
nb = NaiveBayes()
paramGrid_nb = ParamGridBuilder() \
    .addGrid(nb.smoothing, np.linspace(0.3, 10, 10)) \
    .build()
crossval_nb = CrossValidator(estimator=nb,
                          estimatorParamMaps=paramGrid_nb,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds= 5) 
cvModel_nb = crossval_nb.fit(trainDF)

In [15]:
cvModel_nb.avgMetrics

[0.6905510685747699,
 0.6820628462025027,
 0.6779480784866339,
 0.6751874079487993,
 0.6730258628211953,
 0.6713579439975148,
 0.6698091810411042,
 0.6684394703259965,
 0.6673652151046399,
 0.666350182395984]

In [16]:
predictions_nb = cvModel_nb.transform(testDF)
predictions_nb.select('label', 'prediction').show(30)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       2.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       2.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       2.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       2.0|
|  0.0|       2.0|
|  0.0|       2.0|
|  0.0|       2.0|
+-----+----------+
only showing top 30 rows



In [17]:
predictions_nb.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       1.0|   20|
|  0.0|       1.0|   87|
|  2.0|       2.0|   86|
|  1.0|       1.0|  299|
|  2.0|       0.0|    3|
|  0.0|       0.0|  350|
|  1.0|       2.0|   64|
|  0.0|       2.0|   78|
|  1.0|       0.0|   48|
+-----+----------+-----+



In [18]:
my_eval_nb = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_nb.evaluate(predictions_nb)

0.7686762509335324

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_nb.evaluate(predictions_nb)

0.7246578403902293

In [20]:
my_mc_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_nb.evaluate(predictions_nb)

0.7101449275362319

In [ ]:
# Tokenize DataFrame
tokenizer = Tokenizer(inputCol="text", outputCol="Words")

In [ ]:
# Transform DataFrame
reviewed = tokenizer.transform(df2)
reviewed.show()

+--------+--------------------+------+--------------------+
|   class|                text|length|               Words|
+--------+--------------------+------+--------------------+
| neutral|According to Gran...|   127|[according, to, g...|
| neutral|Technopolis plans...|   190|[technopolis, pla...|
|negative|The international...|   228|[the, internation...|
|positive|With the new prod...|   206|[with, the, new, ...|
|positive|According to the ...|   203|[according, to, t...|
|positive|FINANCING OF ASPO...|   178|[financing, of, a...|
|positive|For the last quar...|   193|[for, the, last, ...|
|positive|In the third quar...|   125|[in, the, third, ...|
|positive|Operating profit ...|   122|[operating, profi...|
|positive|Operating profit ...|   103|[operating, profi...|
|positive|TeliaSonera TLSN ...|   178|[teliasonera, tls...|
|positive|STORA ENSO , NORS...|   164|[stora, enso, ,, ...|
|positive|A purchase agreem...|   207|[a, purchase, agr...|
|positive|Finnish Talentum ...|   163|[f

In [ ]:
from pyspark.ml.feature import CountVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# CountVectorizer( minTF=1.0, minDF=1.0, maxDF=9223372036854775807, vocabSize=262144, binary=False, inputCol="text", outputCol="count_vector")
cv = CountVectorizer()
cv.setInputCol("Words")
# CountVectorizer...
cv.setOutputCol("vectors")
# CountVectorizer...
model = cv.fit(reviewed)
model.setInputCol("Words")
# CountVectorizerModel...
cv_model = model.transform(reviewed).show(truncate=False)

+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class   |text                                                                                                      

In [ ]:
# Remove stop words
stopremove = StopWordsRemover(inputCol="Words", outputCol="filtered")

In [ ]:
# Transform new DataFrame
newFrame = stopremove.transform(reviewed)
newFrame.show()

+--------+--------------------+------+--------------------+--------------------+
|   class|                text|length|               Words|            filtered|
+--------+--------------------+------+--------------------+--------------------+
| neutral|According to Gran...|   127|[according, to, g...|[according, gran,...|
| neutral|Technopolis plans...|   190|[technopolis, pla...|[technopolis, pla...|
|negative|The international...|   228|[the, internation...|[international, e...|
|positive|With the new prod...|   206|[with, the, new, ...|[new, production,...|
|positive|According to the ...|   203|[according, to, t...|[according, compa...|
|positive|FINANCING OF ASPO...|   178|[financing, of, a...|[financing, aspoc...|
|positive|For the last quar...|   193|[for, the, last, ...|[last, quarter, 2...|
|positive|In the third quar...|   125|[in, the, third, ...|[third, quarter, ...|
|positive|Operating profit ...|   122|[operating, profi...|[operating, profi...|
|positive|Operating profit .

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
# tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
# stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
# hashingTF = HashingTF(inputCol="filtered", outputCol='hash_token')
# idf = IDF(inputCol='hash_token', outputCol='idf_token')


In [ ]:
reviewed.show()

+--------+--------------------+------+--------------------+
|   class|                text|length|               Words|
+--------+--------------------+------+--------------------+
| neutral|According to Gran...|   127|[according, to, g...|
| neutral|Technopolis plans...|   190|[technopolis, pla...|
|negative|The international...|   228|[the, internation...|
|positive|With the new prod...|   206|[with, the, new, ...|
|positive|According to the ...|   203|[according, to, t...|
|positive|FINANCING OF ASPO...|   178|[financing, of, a...|
|positive|For the last quar...|   193|[for, the, last, ...|
|positive|In the third quar...|   125|[in, the, third, ...|
|positive|Operating profit ...|   122|[operating, profi...|
|positive|Operating profit ...|   103|[operating, profi...|
|positive|TeliaSonera TLSN ...|   178|[teliasonera, tls...|
|positive|STORA ENSO , NORS...|   164|[stora, enso, ,, ...|
|positive|A purchase agreem...|   207|[a, purchase, agr...|
|positive|Finnish Talentum ...|   163|[f

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['vectors', 'length'], outputCol='features')

In [ ]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, cv, clean_up])

In [ ]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df2)
cleaned = cleaner.transform(df2)

In [ ]:
# Show label and resulting features
cleaned.select(['label','features']).show()

AnalysisException: ignored

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show()

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)